# Task 1 d, Calculating strength reserve factor of all elements

## Please enter your name (daniel, fabian, felix, yannis)

In [1]:
name = "fabian"

## Imports 

In [2]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *


## File Selection 

In [3]:
#Import the stress of the panels  
paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)

paneldf

,Component Name,sigmaXX,sigmaYY,sigmaZZ,sigmaXY,sigmaXZ,sigmaYZ,Load Case
Element ID,,,,,,,,
1,panel1,-66.67,12.750,0,26.25,0,0,Subcase 1 (LC1)
2,panel1,-82.97,20.870,0,27.10,0,0,Subcase 1 (LC1)
3,panel1,-67.63,20.740,0,36.97,0,0,Subcase 1 (LC1)
4,panel2,-97.67,2.415,0,34.52,0,0,Subcase 1 (LC1)
5,panel2,-89.50,25.350,0,27.04,0,0,Subcase 1 (LC1)
...,...,...,...,...,...,...,...,...
26,panel9,-44.75,12.670,0,162.30,0,0,Subcase 3 (LC3)
27,panel9,-52.64,-65.300,0,190.50,0,0,Subcase 3 (LC3)
28,panel10,-36.45,32.350,0,192.30,0,0,Subcase 3 (LC3)


In [4]:
# This will become the new import for the panel stresses
paneldf_v2 = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

# Rename the element column to match
#element_component_df.rename(columns={'Element ID': 'element_id'}, inplace=True)
paneldf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)
paneldf_v2.set_index('Element ID', inplace=True)

element_component_df.set_index('Element ID', inplace=True)


# now add the component column to the paneldf_v2 by matching the element_id
paneldf_v2 = pd.concat([paneldf_v2, element_component_df], axis=1)

paneldf_v2.rename(columns={'Loadcase': 'Load Case', 'XX': 'sigmaXX', 'YY': 'sigmaYY', 'XY': 'sigmaXY', 'ZZ': 'sigmaZZ', 'YZ': 'sigmaYZ', 'ZX': 'sigmaXZ'}, inplace=True)
paneldf_v2.drop(columns=['FileID', 'Step', 'Layer'], inplace=True)

# Rename the load case column to match the old one
paneldf_v2['Load Case'] = paneldf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

# copy the new df to the prev.
paneldf = paneldf_v2
#print(paneldf.to_string())
#print(paneldf_v2.to_string())


In [5]:
# Import the stress of the stringers 
stringerdf = pd.read_excel(f'../data/{name}/stringer.xlsx', index_col=0)
print(stringerdf.head(10))

           Component Name  sigmaXX        Load Case
Element ID                                         
40              stringer1   -77.67  Subcase 1 (LC1)
41              stringer1   -70.60  Subcase 1 (LC1)
42              stringer1   -79.68  Subcase 1 (LC1)
43              stringer2   -81.30  Subcase 1 (LC1)
44              stringer2   -85.24  Subcase 1 (LC1)
45              stringer2   -85.16  Subcase 1 (LC1)
46              stringer3   -81.42  Subcase 1 (LC1)
47              stringer3   -83.91  Subcase 1 (LC1)
48              stringer3   -85.17  Subcase 1 (LC1)
49              stringer4   -69.43  Subcase 1 (LC1)


## Constants 

In [6]:
sigma_ult = 530

## Applying Functions 

In [7]:
#Adding reserve Factor column to the panels 
paneldf['reserveFactor'] = paneldf.apply(panelStrength_calc, sigma_ult= sigma_ult, axis=1)
paneldf.head(10)

,Load Case,sigmaXX,sigmaXY,sigmaYY,sigmaYZ,sigmaXZ,sigmaZZ,Component Name,reserveFactor
Element ID,,,,,,,,,
1,Subcase 1 (LC1),-66.67,26.25,12.75,0.0,0.0,0.0,panel1,4.073
2,Subcase 1 (LC1),-82.97,27.10,20.87,0.0,0.0,0.0,panel1,3.331
3,Subcase 1 (LC1),-67.63,36.97,20.74,0.0,0.0,0.0,panel1,3.447
4,Subcase 1 (LC1),-97.67,34.52,2.42,0.0,0.0,0.0,panel2,3.057
5,Subcase 1 (LC1),-89.50,27.04,25.35,0.0,0.0,0.0,panel2,3.085
6,Subcase 1 (LC1),-96.29,28.46,26.60,0.0,0.0,0.0,panel2,2.888
7,Subcase 1 (LC1),-94.14,33.47,-2.96,0.0,0.0,0.0,panel3,3.232
8,Subcase 1 (LC1),-96.16,26.79,20.02,0.0,0.0,0.0,panel3,3.016
9,Subcase 1 (LC1),-93.57,29.76,37.30,0.0,0.0,0.0,panel3,2.768


In [8]:
#Adding reserve Factor column to the stringers 
stringerdf['reserveFactor'] = stringerdf.apply(stringerStrength_calc, sigma_ult= sigma_ult, axis=1)
stringerdf.head(10)

,Component Name,sigmaXX,Load Case,reserveFactor
Element ID,,,,
40,stringer1,-77.67,Subcase 1 (LC1),4.549
41,stringer1,-70.60,Subcase 1 (LC1),5.005
42,stringer1,-79.68,Subcase 1 (LC1),4.434
43,stringer2,-81.30,Subcase 1 (LC1),4.346
44,stringer2,-85.24,Subcase 1 (LC1),4.145
45,stringer2,-85.16,Subcase 1 (LC1),4.149
46,stringer3,-81.42,Subcase 1 (LC1),4.340
47,stringer3,-83.91,Subcase 1 (LC1),4.211
48,stringer3,-85.17,Subcase 1 (LC1),4.149


# Making things pretty 

## Delete unnecessary columns 

In [9]:
# First for the stringers 
stringerdf = stringerdf.drop(['Component Name','sigmaXX'], axis=1)
stringerdf

,Load Case,reserveFactor
Element ID,,
40,Subcase 1 (LC1),4.549
41,Subcase 1 (LC1),5.005
42,Subcase 1 (LC1),4.434
43,Subcase 1 (LC1),4.346
44,Subcase 1 (LC1),4.145
...,...,...
62,Subcase 3 (LC3),8.290
63,Subcase 3 (LC3),11.766
64,Subcase 3 (LC3),7.788


In [10]:
# Then also the panels 
paneldf=paneldf.drop(['Component Name', 'sigmaXX', 'sigmaYY', 'sigmaZZ', 'sigmaXY', 'sigmaXZ', 'sigmaYZ'], axis = 1)
paneldf.head(10)

,Load Case,reserveFactor
Element ID,,
1,Subcase 1 (LC1),4.073
2,Subcase 1 (LC1),3.331
3,Subcase 1 (LC1),3.447
4,Subcase 1 (LC1),3.057
5,Subcase 1 (LC1),3.085
6,Subcase 1 (LC1),2.888
7,Subcase 1 (LC1),3.232
8,Subcase 1 (LC1),3.016
9,Subcase 1 (LC1),2.768


## Concat data frames beneth each other 

In [11]:
combined = pd.concat([paneldf,stringerdf],axis=0)
combined.sample(20)

,Load Case,reserveFactor
Element ID,,
56,Subcase 1 (LC1),4.253
57,Subcase 2 (LC2),8.175
59,Subcase 2 (LC2),6.747
4,Subcase 1 (LC1),3.057
47,Subcase 3 (LC3),8.423
63,Subcase 3 (LC3),11.766
13,Subcase 2 (LC2),4.284
55,Subcase 3 (LC3),2.854
50,Subcase 2 (LC2),6.761


## Split into three seperate loadcase data frames 

In [12]:
loadCase1df = combined[combined['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = combined[combined['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = combined[combined['Load Case'] == 'Subcase 3 (LC3)']
loadCase1df.head(20)

,Load Case,reserveFactor
Element ID,,
1,Subcase 1 (LC1),4.073
2,Subcase 1 (LC1),3.331
3,Subcase 1 (LC1),3.447
4,Subcase 1 (LC1),3.057
5,Subcase 1 (LC1),3.085
6,Subcase 1 (LC1),2.888
7,Subcase 1 (LC1),3.232
8,Subcase 1 (LC1),3.016
9,Subcase 1 (LC1),2.768


In [13]:
# Drop the loadcase columns 
loadCase1df = loadCase1df.drop(['Load Case'], axis = 1)
loadCase2df = loadCase2df.drop(['Load Case'], axis = 1)
loadCase3df = loadCase3df.drop(['Load Case'], axis = 1)
#Rename the reserveFactor column 
loadCase1df = loadCase1df.rename(columns={"reserveFactor": "Load Case 1 RF"})
loadCase2df = loadCase2df.rename(columns={"reserveFactor": "Load Case 2 RF"})
loadCase3df = loadCase3df.rename(columns={"reserveFactor": "Load Case 3 RF"})
loadCase2df.tail(5)

,Load Case 2 RF
Element ID,
62,6.548
63,6.892
64,6.140
65,5.985
66,6.285


In [14]:
# Now we concat them back together 
outputdf = pd.concat([loadCase1df,loadCase2df, loadCase3df], axis=1)
outputdf

,Load Case 1 RF,Load Case 2 RF,Load Case 3 RF
Element ID,,,
1,4.073,6.908,1.087
2,3.331,5.930,1.237
3,3.447,6.019,1.044
4,3.057,5.175,1.054
5,3.085,5.601,1.236
6,2.888,5.079,1.032
7,3.232,5.305,1.026
8,3.016,5.299,1.246
9,2.768,4.933,0.974


# Export back to Excel 

In [15]:
outputdf.to_excel(f'../data/{name}/output/processed.xlsx')